# 動的計画法

## 初めに

- 関数の最大、最小値
  - 一般には全組合せの調査必要
- 関数が2変数の和の場合
  - 最適解の効率的な解法 ＝ 動的計画法

In [2]:
import numpy as np
def Frog1(footholds):
    one_diffs = [abs(f0 - f1) for f0, f1 in zip(footholds, footholds[1:])]
    two_diffs = [abs(f0 - f2) for f0, f2 in zip(footholds, footholds[2:])]
    indexes = [[0]]
    scores = np.zeros((1,))
    indexes.append(indexes[-1] + [len(scores)])
    scores = np.append(scores, one_diffs.pop(0))
    for diff in zip(two_diffs, one_diffs):
        score = np.array([scores[-2] + diff[-2], scores[-1] + diff[-1]])
        i = np.argmin(score) - 2
        indexes.append(indexes[i] + [len(scores)])
        scores = np.append(scores, score[i])
    return indexes[-1]

In [13]:
footholds = [2, 9, 4, 5, 1, 6]
Frog1(footholds)

[0, 2, 3, 5]